In [1]:
# https://www.promodel.com/pdf/DES-ANN-Article-8-20-2018.pdf
# Step one goal: create an ANN and find import variables, w

In [2]:
import matplotlib as matplot
import seaborn
import bokeh
#import keras
import scipy
import plotly
import plotly.express as pe
import numpy
import pandas as pd
from get_tree_data import ncs1_data
import re
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
# import block

In [3]:
ncs1 = ncs1_data()
  # ncs1 holds all of the data from NCS 006693
  # Includes ncs1.dxdm (DS0002 DataFrame) and ncs1.survey (DS001 DataFrame)
  # Key Functions:
    # ncs1.search_for_description looks for a descriotion of a column name in ncs1.dxdm or ncs1.survey
  # Key Variables:
    # ncs1.dxdm (DS0002)
    # ncs1.survey (DS0001)
    # ncs1.tree (Tree including descriptions of survey and dxdm columns)
    # ncs1.root (root used in traversing ncs1.tree)

In [40]:
keras_input = ncs1.survey.iloc[:, ncs1.root.loc[2,"Start"]+1:ncs1.root.loc[2, "End"]]
keras_input.insert(0, 'CASEID', ncs1.survey.loc[:,'CASEID'], True)

train_input = keras_input.sample(frac = .6)
test_input = keras_input.drop(train_input.index)

keras_output = ncs1.dxdm.loc[:, ('CASEID', 'DRGALT1')]

train_output = keras_output.drop(test_input.index)
test_output = keras_input.drop(train_input.index)

# setup train/test input vars

train_output = train_output.reset_index(drop=True)
test_output = test_output.reset_index(drop=True)

train_input = train_input.reset_index(drop=True)
test_input = test_input.reset_index(drop=True)

In [41]:
train_input

,CASEID,V102,V103,V104,V105,V106,V107,V108,V109,V110,...,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249
0,11025,2,1,2,7,1,0,5,5,0,...,0,0,0,1,1,0,9,2,4,4
1,23759,2,1,2,8,2,0,5,5,0,...,0,0,0,1,2,0,1,3,4,4
2,11463,3,3,1,7,1,0,5,5,0,...,0,0,0,1,2,0,3,3,3,4
3,23333,2,3,5,6,1,0,1,0,1,...,1,1,3,0,0,2,3,1,4,4
4,11408,2,1,2,6,3,0,3,1,0,...,1,1,1,0,0,1,4,1,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4854,25445,2,1,3,8,1,0,1,0,1,...,1,1,3,0,0,2,2,2,2,4
4855,22120,3,1,1,8,1,0,1,0,1,...,1,1,7,0,0,1,1,1,4,4
4856,25474,2,1,5,7,1,0,1,0,1,...,5,0,0,0,0,3,2,3,1,4
4857,11415,5,5,1,10,2,0,3,5,0,...,0,0,0,5,0,0,5,4,1,2


In [42]:
# use sections from ncs1.root and see what keras has to say about them in relation to DRGALT1 
model = Sequential()
model.add(Dense(12, input_dim=12, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(6, activation='sigmoid'))
model.add(Dense(3, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
model.fit(train_input.iloc[:, 1:13], train_output.iloc[:, 1], epochs=150, batch_size=10)

Epoch 1/150
486/486 [==============================] - 1s 1ms/step - loss: 0.7594 - accuracy: 0.3886
Epoch 2/150
486/486 [==============================] - 1s 1ms/step - loss: 0.4286 - accuracy: 0.8811
Epoch 3/150
486/486 [==============================] - 1s 1ms/step - loss: 0.3709 - accuracy: 0.8870
Epoch 4/150
486/486 [==============================] - 1s 1ms/step - loss: 0.3575 - accuracy: 0.8870
Epoch 5/150
486/486 [==============================] - 0s 998us/step - loss: 0.3564 - accuracy: 0.8858
Epoch 6/150
486/486 [==============================] - 0s 1ms/step - loss: 0.3550 - accuracy: 0.8861
Epoch 7/150
486/486 [==============================] - 0s 1ms/step - loss: 0.3596 - accuracy: 0.8837
Epoch 8/150
486/486 [==============================] - 1s 1ms/step - loss: 0.3584 - accuracy: 0.8842
Epoch 9/150
486/486 [==============================] - 1s 1ms/step - loss: 0.3572 - accuracy: 0.8849
Epoch 10/150
486/486 [==============================] - 1s 1ms/step - loss: 0.3437 - accu

In [37]:
_, accuracy = model.evaluate(train_input.iloc[:, 1:13], train_output.iloc[:, 1])

152/152 [==============================] - 0s 974us/step - loss: 0.3561 - accuracy: 0.8854


In [38]:
predictions = model.predict_classes(test_input.iloc[:, 1:13])

In [39]:
right = 0
for i in range(0, len(predictions)):
    if predictions[i] == test_output.iloc[i, 1]:
        right = right+1
print("accuracy: " + str(right/len(test_output)))

accuracy: 0.0
